# 6장: 예지력(단계별 사고)

- [수업](#lesson)
- [연습 문제](#exercises)
- [예시 놀이터](#example-playground)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 헬퍼 함수를 설정합니다.

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None, prefill=None):
    inference_config = {
        "temperature": 0.0,
        "maxTokens": 200
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config,
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    if prefill:
        converse_api_params["messages"].append({"role": "assistant", "content": [{"text": prefill}]})
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## 수업

만약 누군가가 당신을 깨워 바로 복잡한 질문을 여러 개 해야 한다면 어떻게 대답할 것인가? 아마도 **먼저 답변을 생각해볼 시간**이 주어진다면 그렇지 않을 때보다 잘할 것이다.

Claude도 마찬가지다.

**Claude에게 단계별로 생각할 시간을 주면 특히 복잡한 작업에서 더 정확해질 수 있다**. 하지만 **생각하는 것은 소리 내어야만 한다**. Claude에게 생각하라고 하면서 답변만 출력하라고 하면 실제로는 생각하지 않은 것이다.

### 예시

아래 프롬프트에서 두 번째 문장이 첫 번째 문장을 부정하는 것은 사람에게는 명확하다. 그러나 **Claude는 "unrelated"라는 단어를 너무 문자 그대로 받아들인다**.

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

Claude의 응답을 개선하기 위해서는 **Claude에게 먼저 생각할 시간을 주자**. 그렇게 하려면 Claude가 작업을 처리하고 생각하기 위해 취해야 할 단계를 문자 그대로 적어주면 된다. 역할 프롬프팅을 약간 더하면 Claude가 리뷰를 더 깊이 이해할 수 있게 된다.

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Claude는 때때로 순서에 민감하다**. 이 예시는 Claude가 미묘한 텍스트를 이해하는 능력의 한계에 있으며, 앞의 예시에서 부정적인 것과 긍정적인 것의 순서를 바꾸면 Claude의 전반적인 평가가 긍정적으로 바뀐다.

대부분의 상황에서(하지만 혼란스럽게도 모든 상황은 아니다) **Claude는 두 가지 옵션 중 두 번째 것을 선택할 가능성이 더 높다**. 아마도 웹에서 수집한 학습 데이터에서 두 번째 옵션이 맞는 경우가 더 많았기 때문일 것이다.

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

**Claude에게 생각할 시간을 주면 Claude의 답변이 잘못된 것에서 올바른 것으로 바뀔 수 있다**. 많은 경우에 그렇게 간단하다!

Claude의 답변이 잘못된 예시를 살펴보며 Claude에게 생각하라고 하면 어떻게 고칠 수 있는지 알아보자.

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

이를 해결하기 위해 `<brainstorm>` 태그를 사용하여 Claude에게 단계별로 생각하라고 요청해보자.

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))

위의 내용을 변경하지 않고 수업 프롬프트를 실험해보고 싶다면 수업 노트북의 맨 아래로 스크롤하여 [**예시 놀이터**](#example-playground)를 방문하세요.

---

## 연습 문제
- [연습문제 6.1 - 이메일 분류하기](#exercise-61---classifying-emails)
- [연습문제 6.2 - 이메일 분류 형식화](#exercise-62---email-classification-formatting)

### 연습 6.1 - 이메일 분류하기
이 연습에서는 Claude에게 다음 범주로 이메일을 분류하도록 지시할 것입니다:										
- (A) 판매 전 문의
- (B) 고장 또는 결함 제품
- (C) 청구 문의
- (D) 기타(설명 필요)

연습의 첫 번째 부분에서는 `PROMPT`를 변경하여 **Claude가 올바른 분류를 출력하고 분류만을 출력**하도록 해야 합니다. 답변에는 **올바른 선택지의 문자(A - D)와 괄호, 그리고 범주 이름이 포함**되어야 합니다.

`EMAILS` 목록의 각 이메일 옆에 있는 주석을 참조하여 해당 이메일이 어떤 범주로 분류되어야 하는지 알아보세요.

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ 힌트가 필요하다면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_6_1_hint)

아직도 막혔나요? 아래 셀을 실행하면 예시 해답을 볼 수 있습니다.						

In [ ]:
print(hints.exercise_6_1_solution)

### 연습 6.2 - 이메일 분류 포맷팅
이번 연습에서는 위의 프롬프트 출력을 원하는 대로 포맷팅할 것입니다. 

가장 좋아하는 출력 포맷팅 기법을 사용하여 Claude가 올바른 분류의 문자만 `<answer></answer>` 태그로 감싸도록 하세요. 예를 들어, 첫 번째 이메일에 대한 답변에는 `<answer>B</answer>`라는 정확한 문자열이 포함되어야 합니다.

`EMAILS` 목록의 각 이메일 옆에 있는 주석을 참조하여 각 이메일에 대한 올바른 문자 범주를 잊지 않도록 하세요.

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ 힌트가 필요하다면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_6_2_hint)

### 축하합니다!

지금까지의 모든 연습을 해결했다면 다음 장으로 넘어갈 준비가 되었습니다. 프롬프팅을 즐기세요!

---

## 예제 놀이터

여기는 이 강의에서 보여준 프롬프트 예제를 자유롭게 실험하고 프롬프트를 조정하여 Claude의 응답이 어떻게 달라지는지 확인할 수 있는 공간입니다.

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))